# 겐심 튜토리얼
- https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-lee.ipynb

In [ ]:
# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
lee_test_file = test_data_dir + os.sep + 'lee.cor'

In [3]:
import gensim
import os
import collections
import smart_open
import random

### 파일 읽는 함수

In [1]:
def read_corpus(fname, tokens_only = False):
    with smart_open.smart_open(fname, encoding = "utf-8") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

### 훈련파일 / 테스트파일 열기 인데 난 테스트 안하니까 wiki+news열기

In [20]:
train_corpus_wiki = list(read_corpus('sample.txt'))
test_corpus_wiki = list(read_corpus('test.txt'))

### 모델 파라미터 설정

In [7]:
model = gensim.models.doc2vec.Doc2Vec(vector_size = 50, min_count = 2, epochs = 40)

### 모델 vocab 학습

In [9]:
model.build_vocab(train_corpus_wiki)

### 모델 학습 (시간 재면서)

In [10]:
%time model.train(train_corpus_wiki, total_examples = model.corpus_count, epochs = model.epochs)

Wall time: 6.62 s


### Test : 학습된 모델에 문장 넣고 벡터값 뽑기

In [14]:
model.infer_vector(['노무현','김구'])

array([ 0.07913499,  0.07671497,  0.06181526, -0.033435  , -0.033475  ,
       -0.1884527 , -0.0136215 , -0.15140074, -0.03188309, -0.01196146,
        0.05427239, -0.06935588, -0.02579572,  0.05812091,  0.16795143,
       -0.0202156 , -0.04447793, -0.00703916,  0.01623327, -0.10363165,
        0.00804774, -0.01608106,  0.00135193, -0.0758067 , -0.05634179,
        0.00500879,  0.05876822, -0.01870101,  0.04947336, -0.03054612,
       -0.06647369, -0.08252011,  0.03508627,  0.02662414,  0.10805817,
       -0.01307449,  0.01947933, -0.03501626,  0.0551618 ,  0.07311437,
       -0.099995  , -0.0838246 ,  0.00364303, -0.05054481,  0.18556993,
       -0.01375187,  0.05845632,  0.08266502, -0.00262982,  0.0226048 ],
      dtype=float32)

In [11]:
train_corpus_wiki

[TaggedDocument(words=['문재인'], tags=[0]),
 TaggedDocument(words=['최근', '수정', '시각'], tags=[1]),
 TaggedDocument(words=[], tags=[2]),
 TaggedDocument(words=[], tags=[3]),
 TaggedDocument(words=[], tags=[4]),
 TaggedDocument(words=['분류', '문재인'], tags=[5]),
 TaggedDocument(words=['파일', '나무위키', '유도', 'png', '동명이인에', '대한', '내용은', '문재인', '동명이인', '문서를', '참조하십시오'], tags=[6]),
 TaggedDocument(words=['파일', '나무위키', '하위문서', 'png', '하위', '문서', '문재인', '생애', '문재인', '평가', '문재인', '다른', '정치인들과의', '관계'], tags=[7]),
 TaggedDocument(words=['파일', 'semi_protect', 'png'], tags=[8]),
 TaggedDocument(words=['문서는', '비로그인', '사용자', '가입', '일이', '지나지', '않은', '일반', '사용자의', '편집이', '제한되어', '있습니다', '자세한', '사유는', '여기를', '참고하시기', '바랍니다'], tags=[9]),
 TaggedDocument(words=['파일', 'px', 'svg', 'png'], tags=[10]),
 TaggedDocument(words=['대한민국', '대통령'], tags=[11]),
 TaggedDocument(words=['펼치기', '접기'], tags=[12]),
 TaggedDocument(words=['역임한', '직위'], tags=[13]),
 TaggedDocument(words=['펼치기', '접기'], tags=[14]),
 TaggedDocument(wo

In [15]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus_wiki)):
    inferred_vector = model.infer_vector(train_corpus_wiki[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

In [16]:
collections.Counter(ranks)

Counter({8: 13,
         90: 9,
         153: 5,
         243: 3,
         2005: 1,
         58: 7,
         3: 28,
         36: 7,
         21: 12,
         105: 8,
         51: 10,
         11: 11,
         608: 1,
         612: 1,
         622: 2,
         9: 23,
         26: 12,
         499: 1,
         87: 4,
         855: 1,
         158: 5,
         634: 1,
         1994: 1,
         112: 6,
         638: 1,
         508: 1,
         899: 1,
         797: 1,
         564: 1,
         520: 2,
         629: 2,
         32: 13,
         20: 13,
         114: 6,
         2125: 1,
         709: 2,
         742: 1,
         104: 7,
         0: 42,
         107: 12,
         5: 19,
         1045: 1,
         507: 1,
         340: 1,
         804: 2,
         193: 5,
         596: 1,
         916: 2,
         552: 1,
         660: 2,
         1009: 1,
         183: 2,
         16: 13,
         79: 8,
         102: 6,
         77: 8,
         234: 1,
         42: 12,
         12: 14,
  

In [17]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus_wiki[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus_wiki[sims[index][0]].words)))

Document (2147): «한신대학교에서 주는 상이다 한신대학교는 문익환 목사가 교수로 재직했던 한국기독교장로회 학교이고 장준하 본인 또한 강제 징집 때문에 마치지 못한 신학 공부를 한신대에서 수학하며 끝낸 있다 한신대는 학생부터 교직원까지 전반적으로 민주화 운동에 앞장섰기에 문익환 장준하와 덤으로 그들의 친구였던 시인 윤동주까지 합쳐 셋을 모두 자신들 학교의 정신적 지주이자 시발점으로 보고 있다»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3):

MOST (1843, 0.9817487597465515): «해당 내용에 대해서 보충하자면 김구와 대한민국 임시정부로 대표되는 지지 세력들은 일제강점기 당시에는 다양한 항일투쟁 조직을 결성하면서 적극적으로 저항했으며 한국 독립운동의 구심점이었다고 있다 이를 통해서 대중들로 하여금 커다란 지지를 얻었던 것이다 그러나 최근에 진행된 연구에 따르면 김구 세력의 대일관은 명확한 반일이었으나 해방 이후 본격적으로 언급되기 시작한 친일파 처리 문제에 대해서는 상당히 소극적이었다고 한다 김구 세력의 이러한 태도는 상대적으로 적극적인 친일파 청산을 주장한 박헌영 계열의 좌익 세력들에게 많은 비판을 받게 되었으며 대중의 지지도 줄어들게 만드는 계기가 되었다 물론 처음부터 김구 세력의 전체가 이와 같은 입장을 취한 것은 아닌데 김원봉 김성숙 김주식 좌익 민족주의 계열 인사들은 친일파 청산에 있어서 강경한 입장을 취하고 있었다 그러나 김구 조소앙 신익희 주류를 형성하고 있던 우파 인사들은 친일파 청산 문제에 소극적이었으며 먼저 민족통일을 이루고 뒤에 친일파를 청산해야 한다는 태도를 고수했다 그러한 논리는 급속도로 김구 세력 전체의 입장으로 고착화되었다 김구 세력이 해방 이전의 입장에서 상당히 후퇴한 태도를 견지하게 이유로는 여러 가지가 있는데 번째로 대중적 인지도는 높았으나 국내에서의 정치적 기반은 취약했기 때문에 많은 활동을 위하여 이승만과 한민

In [19]:
# Pick a random document from the corpus and infer a vector from the model
doc_id = random.randint(0, len(train_corpus_wiki) - 1)

# Compare and print the second-most-similar document
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus_wiki[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(train_corpus_wiki[sim_id[0]].words)))

Train Document (1589): «독립운동가를 대상으로 테러 사주»

Similar Document (769, 0.9620089530944824): «http archives knowhow or kr record all view page»



In [21]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus_wiki) - 1)
inferred_vector = model.infer_vector(test_corpus_wiki[doc_id])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus_wiki[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus_wiki[sims[index][0]].words)))

TypeError: sequence item 0: expected str instance, list found